In [1]:
import pandas as pd
import numpy as np

In [2]:
script_df = pd.read_csv('script.csv')

In [3]:
script_lab = script_df[['cast','dialog']]

In [4]:
#Filter for only "main" characters (easier for classification) based on counts in dataset 
script_cnts = script_lab.groupby('cast').count().sort_values(by='dialog',ascending=False)
script_cnts.head()


,dialog
cast,
Frasier,21088
Niles,10178
Martin,7629
Daphne,5243
Roz,4735


In [5]:
#Do the filtering
min_samples = 1000
main_chars=script_cnts[script_cnts['dialog']>=min_samples].index
script_filt = script_lab[script_lab['cast'].isin(main_chars)].copy()

In [6]:
# fix bracketed actions. 
#script_filt.loc[:,['dialog']] = 
script_filt['dialog'] = script_filt['dialog'].str.replace(("\[.*\]"),'')

In [53]:
print(main_chars)

Index(['Frasier', 'Niles', 'Martin', 'Daphne', 'Roz'], dtype='object', name='cast')


In [27]:
script_filt = script_filt.dropna()

In [28]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = script_filt['dialog']

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)

In [33]:
X

<48873x20256 sparse matrix of type '<class 'numpy.int64'>'
	with 517068 stored elements in Compressed Sparse Row format>

In [64]:
Y = pd.get_dummies(script_filt['cast'])

In [65]:
from sklearn.model_selection import train_test_split 
xtrain, xtest, ytrain, ytest = train_test_split(X,Y,test_size=.2,random_state=1930)


In [66]:
num_training, num_features = xtrain.get_shape()

In [91]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(units=5000, activation='sigmoid',input_shape=(num_features,)))
#model.add(Dense(units=4200, activation='sigmoid'))
model.add(Dense(units=780, activation='softmax'))
model.add(Dense(units=120, activation='tanh'))
model.add(Dense(units=5, activation='relu'))



In [92]:
model.compile(loss='categorical_crossentropy',optimizer='sgd',metrics=['accuracy'])

In [93]:
model.fit(xtrain,ytrain,epochs=1,batch_size=32)

Epoch 1/1
39098/39098 [==============================] - 1303s 33ms/step - loss: 4.9579 - acc: 0.4306


In [94]:
model.evaluate(xtest,ytest,batch_size=128)

9775/9775 [==============================] - 40s 4ms/step


[4.865193110512346, 0.43508951413357044]